<h1> Exercicio 6 </h1>
By: Luan C. Klein

In [1]:
import pandas as pd
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import HeatMap

In [2]:
df = pd.read_csv("X_trainToronto.csv")

a) Agrupe os estabelecimentos que são populares. Varie os valores de eps e min_samples. Escolha
uma combinação que julgue produzir um resultado interessante. Justifique a sua escolha.

In [3]:
print("POPULARES")
df_populares = df[df['destaque'] == 1]
df_populares = df_populares[['latitude', 'longitude']]

mapaToronto = folium.Map(location=[43.651070, -79.347015])

dbscanConfig = DBSCAN(eps=0.001, min_samples=40)
## Esses valores foram escolhidos porque separam em 3 regiões os estabelecimentos de destaque. Uma delas é bastante 'forte'
## enquanto que as outras duas são um pouco mais 'fracas'.

dbscanResults = dbscanConfig.fit(df_populares)
dbscanResults.labels_
df_populares_filtered = df_populares[dbscanResults.labels_ !=-1]

pontos  = [[row[0],row[1]] for index, row in df_populares_filtered.iterrows()]

HeatMap(pontos).add_to(mapaToronto)
mapaToronto

POPULARES


b) Agrupe os estabelecimentos que não são populares. Varie os valores de eps e min_samples.
Escolha uma combinação que julgue produzir um resultado interessante. Justifique a sua escolha.

In [4]:
import folium
from folium.plugins import HeatMap
print("NÃO POPULARES")
df_nao_populares = df[df['destaque'] == 0]
df_nao_populares = df_nao_populares[['latitude', 'longitude']]
mapaToronto = folium.Map(location=[43.651070, -79.347015])

dbscanConfig = DBSCAN(eps=0.0008, min_samples=50)
## Esses valores foram escolhidos porque separam os estabelecimentos de não destaque em duas regiões distintas, uma mais 'forte' e outra mais 'fraca'.
dbscanResults = dbscanConfig.fit(df_nao_populares)
dbscanResults.labels_
df_nao_populares_filtered = df_nao_populares[dbscanResults.labels_ !=-1]

pontos  = [[row[0],row[1]] for index, row in df_nao_populares_filtered.iterrows()]

HeatMap(pontos).add_to(mapaToronto)
mapaToronto

NÃO POPULARES


c) Visualize os clusters encontrados para ambos os casos. Existe alguma tendência de concentração
em determinadas partes da cidade? Faça uma breve análise dos resultados encontrados.

Sim, existe uma tendencia. Observa-se que a região ao lado da spadina avenue é uma região de grande concentração de estabecimentos
de destaque. 
Já a região perto de Yonge street apresenta uma concetação de estabelecimentos de não destaque.
Além disso, a região perto de Yorkville Street apresenta uma concetração tanto de estabelecimentos de destaque como de não destaque.

In [5]:
import numpy as np

In [6]:
def dbscan_predict(model, X):
    nr_samples = X.shape[0]
    y_new = np.ones(shape=nr_samples, dtype=int) * -1
    for i in range(nr_samples):
        diff = model.components_ - list(X.iloc[i, :])
        dist = np.linalg.norm(diff, axis=1)  # Euclidean distance
        shortest_dist_idx = np.argmin(dist)
        if dist[shortest_dist_idx] < model.eps:
            y_new[i] = model.labels_[model.core_sample_indices_[shortest_dist_idx]]
    return y_new

In [14]:
df_teste = pd.read_csv("X_testToronto.csv")
df = pd.read_csv("X_trainToronto.csv")
ids = df['business_id']
df_use = df[['latitude', 'longitude']]

In [8]:
result_not_dest = dbscan_predict(dbscanResults, df_use)

In [9]:
list_tuple = list(zip(ids, result_not_dest))
df_final = pd.DataFrame(list_tuple, columns=['business_id', 'no_dest_zone'])
df_final['no_dest_zone'].describe()

count    2836.000000
mean       -0.966150
std         0.271355
min        -1.000000
25%        -1.000000
50%        -1.000000
75%        -1.000000
max         2.000000
Name: no_dest_zone, dtype: float64

In [26]:
df_teste = pd.read_csv("X_testToronto.csv")
df = pd.read_csv("X_trainToronto.csv")

In [70]:
from sklearn.cluster import DBSCAN

def add_features_places_train(df):
    df_use = df.copy()
    ids = df_use['business_id']
    
    df_pop = df_use[df_use['destaque'] == 1]
    ids_pop = df_pop['business_id']
    df_no_pop = df_use[df_use['destaque'] == 0]
    ids_no_pop = df_pop['business_id']

    df_pop = df_pop[['latitude', 'longitude']]
    df_no_pop = df_no_pop[['latitude', 'longitude']]
    
    dbscanConfig_pop = DBSCAN(eps=0.001, min_samples=40)
    dbscanConfig_no_pop = DBSCAN(eps=0.0008, min_samples=50)

    
    dbscanResults_pop = dbscanConfig_pop.fit(df_pop)
    dbscanResults_no_pop = dbscanConfig_no_pop.fit(df_no_pop)
    
    list_pop = list(zip(ids_pop, dbscanResults_pop.labels_))
    df_pop_result = pd.DataFrame(list_pop, columns=['business_id', 'pop_zone'])
    
    list_no_pop = list(zip(ids_no_pop, dbscanResults_no_pop.labels_))
    df_no_pop_result = pd.DataFrame(list_pop, columns=['business_id', 'no_pop_zone'])
    
    df = df.merge(df_pop_result, left_on='business_id', right_on='business_id', how='left')
    df = df.merge(df_no_pop_result, left_on='business_id', right_on='business_id', how='left')
    
    
    df['pop_zone'] = df['pop_zone'].fillna(-1)
    df['no_pop_zone'] = df['no_pop_zone'].fillna(-1)
    
    return df, dbscanResults_pop, dbscanResults_no_pop

def add_features_places_test(df, model_pop, model_no_pop):
    df_use =df.copy()
    ids = df_use['business_id']
    df_use = df_use[['latitude', 'longitude']]
    def dbscan_predict(model, X):
        nr_samples = X.shape[0]
        y_new = np.ones(shape=nr_samples, dtype=int) * -1
        for i in range(nr_samples):
            diff = model.components_ - list(X.iloc[i, :])
            dist = np.linalg.norm(diff, axis=1)  # Euclidean distance
            shortest_dist_idx = np.argmin(dist)
            if dist[shortest_dist_idx] < model.eps:
                y_new[i] = model.labels_[model.core_sample_indices_[shortest_dist_idx]]
        return y_new
    pred_zone_pop = dbscan_predict(model_pop, df_use)
    pred_zone_no_pop = dbscan_predict(model_no_pop, df_use)
    df['pop_zone'] = pred_zone_pop
    df['no_pop_zone'] = pred_zone_no_pop
    return df

In [65]:
from sklearn.model_selection import train_test_split
df_train, df_teste = train_test_split(df, test_size=0.2, random_state=42)

In [72]:
df_exit, dbscanResults_pop, dbscanResults_no_pop = add_features_places_train(df_train)

In [67]:
saida = add_features_places_test(df_teste, dbscanResults_pop, dbscanResults_no_pop)

In [68]:
saida['destaque'].corr(saida['no_pop_zone'])

-0.03842158019734624

In [69]:
saida['destaque'].corr(saida['pop_zone'])

0.02966054352400128